## **Introduction to XGBoost**

by _Titipat Achakulvisut_

In this tutorial, I use all examples and materials from [ParrotPrediction/docker-course-xgboost](https://github.com/ParrotPrediction/docker-course-xgboost). See the full slides of this presentation [here](http://kordinglab.com/lab_teaching_2016/session_2/)

Thanks [Norbert](https://github.com/khozzy) to allow using his materials!

## **Install XGBoost**

After install `XGBoost`, we'll print out version of `xgboost` and `scikit-learn`

In [1]:
import os

In [2]:
import xgboost as xgb
print(xgb.__version__)

0.6


In [3]:
import sklearn
print(sklearn.__version__)

0.18


## **Single decision tree**

In [4]:
from sklearn.datasets import make_classification
from sklearn.cross_validation import train_test_split
seed = 104

/Users/RoozbehFarhoudi/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
X, y = make_classification(n_samples=1000, n_features=20, 
                           n_informative=8, n_redundant=3, 
                           n_repeated=2, random_state=seed)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import log_loss, accuracy_score
from IPython.display import Image

decision_tree = DecisionTreeClassifier(random_state=seed)
decision_tree.fit(X_train, y_train)

y_pred  = decision_tree.predict(X_test)
y_pred_prob  = decision_tree.predict_proba(X_test)

# evaluation
accuracy = accuracy_score(y_test, y_pred)
logloss = log_loss(y_test, y_pred_prob)

In [8]:
print("Single Decision Tree")
print(accuracy)
print(logloss)
print(decision_tree.tree_.node_count) # number of node created

Single Decision Tree
0.78
7.59853080688
167


In [9]:
import os
from sklearn.tree import export_graphviz
dot_file = os.path.join(os.getcwd(), 'images', 'tree.dot')
export_graphviz(decision_tree, out_file=dot_file)

## **Visualize decision tree**

```bash
brew install graphviz
dot -Tpng tree.dot -o tree.png # visualize tree
```

## **AdaBoost**

We'll run Adaboost classifier with 1000 stump trees (only one decision node)

In [15]:
from sklearn.ensemble import AdaBoostClassifier
adaboost = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=2),
    algorithm='SAMME',
    n_estimators=1000,
    random_state=seed)
adaboost.fit(X_train, y_train)
y_pred = adaboost.predict(X_test)
y_pred_prob = adaboost.predict_proba(X_test)

accuracy = accuracy_score(y_test, y_pred)
logloss = log_loss(y_test, y_pred_prob)

print("== AdaBoost ==")
print(accuracy)
print(logloss)

== AdaBoost ==
0.78
0.687665259874


In [ ]:
print(len(adaboost.estimators_))
print(adaboost.estimators_[0]) # estimator 

## **Gradient Boosted Trees**

We'll construct gradient boosted tree with 1000 estimators (trees)

In [13]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(
    max_depth=1,
    n_estimators=1000,
    warm_start=False,
    random_state=seed)
gbc.fit(X_train, y_train)

# make predictions
y_pred = gbc.predict(X_test)
y_pred_prob = gbc.predict_proba(X_test)

# calculate log loss
gbc_accuracy = accuracy_score(y_test, y_pred)
gbc_logloss = log_loss(y_test, y_pred_prob)

print("== Gradient Boosting ==")
print(gbc_accuracy)
print(gbc_logloss)

== Gradient Boosting ==
0.81
0.480566182642


## **XGBoost Standard interface**

In [16]:
import numpy as np
import xgboost as xgb

In [17]:
dtrain = xgb.DMatrix('data/agaricus.txt.train')
dtest = xgb.DMatrix('data/agaricus.txt.test')

In [18]:
params = {
    'objective':'binary:logistic',
    'max_depth':2,
    'silent':1,
    'eta':1
}
num_rounds = 5

In [19]:
bst = xgb.train(params, dtrain, num_boost_round=num_rounds)

we can also observe performace on test dataset using `watchlist` (native interface only)

In [20]:
watchlist  = [(dtest,'test'), (dtrain,'train')]
bst = xgb.train(params, dtrain, num_rounds, watchlist)

[0]	test-error:0.042831	train-error:0.046522
[1]	test-error:0.021726	train-error:0.022263
[2]	test-error:0.006207	train-error:0.007063
[3]	test-error:0.018001	train-error:0.0152
[4]	test-error:0.006207	train-error:0.007063


In [ ]:
preds_prob = bst.predict(dtest)
labels = dtest.get_label()
preds = preds_prob > 0.5

In [ ]:
print("total %i/%i" % (np.sum(labels == preds), len(preds)))

## **XGBoost Scikit-learn interface**

In [ ]:
from xgboost.sklearn import XGBClassifier
from sklearn.datasets import load_svmlight_files

In [ ]:
X_train, y_train, X_test, y_test = load_svmlight_files(('data/agaricus.txt.train', 
                                                        'data/agaricus.txt.test'))

All the parameters are set like in the previous example
- we are dealing with binary classification problem (`'objective':'binary:logistic'`),
- we want shallow single trees with no more than 2 levels (`'max_depth':2`),
- we don't any oupout (`'silent':1`),
- we want algorithm to learn fast and aggressively (`'learning_rate':1`), (in naive named `eta`)
- we want to iterate only 5 rounds (`n_estimators`)

In [ ]:
params = {
    'objective': 'binary:logistic',
    'max_depth': 2,
    'learning_rate': 1.0,
    'silent': 1.0,
    'n_estimators': 5
}

In [ ]:
bst = XGBClassifier(**params).fit(X_train, y_train)

In [ ]:
preds = bst.predict(X_test)

In [ ]:
preds_prob = bst.predict_proba(X_test)

In [ ]:
print("total %i/%i" % (np.sum(labels == preds), len(preds)))

## **Spot most important features**

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
dtrain = xgb.DMatrix('data/agaricus.txt.train')
dtest = xgb.DMatrix('data/agaricus.txt.test')

In [ ]:
params = {
    'objective':'binary:logistic',
    'max_depth':1,
    'silent':1,
    'eta':0.5
}
num_rounds = 5
watchlist  = [(dtest,'test'), (dtrain,'train')]
bst = xgb.train(params, dtrain, num_rounds, watchlist)

to see which features provided the most gain

In [ ]:
xgb.plot_importance(bst, importance_type='gain', xlabel='Gain')

> **F-score** - sums up how many times a split was performed on each feature. 

In [ ]:
xgb.plot_importance(bst)

In [ ]:
importances = bst.get_fscore()
print(importances)

## **Bias and Variance**

In [ ]:
from sklearn.datasets import make_classification
from sklearn.cross_validation import StratifiedKFold
from xgboost.sklearn import XGBClassifier
from sklearn.learning_curve import validation_curve
seed = 123

In [ ]:
X, y = make_classification(n_samples=1000, n_features=20, 
                           n_informative=8, n_redundant=3, 
                           n_repeated=2, random_state=seed)

In [ ]:
cv = StratifiedKFold(y, n_folds=10, shuffle=True, random_state=seed)

In [ ]:
default_params = {
    'objective': 'binary:logistic',
    'max_depth': 1,
    'learning_rate': 0.3,
    'silent': 1.0
}

n_estimators_range = np.linspace(1, 200, 10).astype('int')

train_scores, test_scores = validation_curve(
    XGBClassifier(**default_params),
    X, y,
    param_name = 'n_estimators',
    param_range = n_estimators_range,
    cv=cv,
    scoring='accuracy'
)

In [ ]:
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

fig = plt.figure(figsize=(10, 6), dpi=100)

plt.title("Validation Curve with XGBoost (eta = 0.3)")
plt.xlabel("number of trees")
plt.ylabel("Accuracy")
plt.ylim(0.7, 1.1)

plt.plot(n_estimators_range,
             train_scores_mean,
             label="Training score",
             color="r")

plt.plot(n_estimators_range,
             test_scores_mean, 
             label="Cross-validation score",
             color="g")

plt.fill_between(n_estimators_range, 
                 train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, 
                 alpha=0.2, color="r")

plt.fill_between(n_estimators_range,
                 test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std,
                 alpha=0.2, color="g")

plt.axhline(y=1, color='k', ls='dashed')

plt.legend(loc="best")
plt.show()

In [ ]:
i = np.argmax(test_scores_mean)
print(test_scores_mean[i], n_estimators_range[i])

let's do something > use 70 % randomly chosen and 60 % randomly chosen features

In [ ]:
default_params = {
    'objective': 'binary:logistic',
    'max_depth': 2, # changed
    'learning_rate': 0.3,
    'silent': 1.0,
    'colsample_bytree': 0.6, # added
    'subsample': 0.7 # added
}

n_estimators_range = np.linspace(1, 200, 10).astype('int')

train_scores, test_scores = validation_curve(
    XGBClassifier(**default_params),
    X, y,
    param_name = 'n_estimators',
    param_range = n_estimators_range,
    cv=cv,
    scoring='accuracy'
)

In [ ]:
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

fig = plt.figure(figsize=(10, 6), dpi=100)

plt.title("Validation Curve with XGBoost (eta = 0.3)")
plt.xlabel("number of trees")
plt.ylabel("Accuracy")
plt.ylim(0.7, 1.1)

plt.plot(n_estimators_range,
             train_scores_mean,
             label="Training score",
             color="r")

plt.plot(n_estimators_range,
             test_scores_mean, 
             label="Cross-validation score",
             color="g")

plt.fill_between(n_estimators_range, 
                 train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, 
                 alpha=0.2, color="r")

plt.fill_between(n_estimators_range,
                 test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std,
                 alpha=0.2, color="g")

plt.axhline(y=1, color='k', ls='dashed')

plt.legend(loc="best")
plt.show()

In [ ]:
i = np.argmax(test_scores_mean)
print(test_scores_mean[i], n_estimators_range[i])

## **Hyper parameters tuning**

first we'll use `GridSearchCV` on `max_depth`, `n_estimators` and `learning_rate`

In [ ]:
from scipy.stats import randint, uniform
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV

In [ ]:
seed = 342
X, y = make_classification(n_samples=1000, n_features=20, 
                           n_informative=8, n_redundant=3, 
                           n_repeated=2, random_state=seed)

In [ ]:
cv = StratifiedKFold(y, n_folds=10, shuffle=True, random_state=seed)

In [ ]:
params_grid = {
    'max_depth': [1, 2, 3],
    'n_estimators': [5, 10, 25, 50],
    'learning_rate': np.linspace(1e-16, 1, 3)
}

params_fixed = {
    'objective': 'binary:logistic',
    'silent': 1
}

In [ ]:
bst_grid = GridSearchCV(
    estimator=XGBClassifier(**params_fixed, seed=seed),
    param_grid=params_grid,
    cv=cv,
    scoring='accuracy'
)

In [ ]:
bst_grid.fit(X, y)

In [ ]:
bst_grid.grid_scores_

In [ ]:
bst_grid.best_score_

In [ ]:
bst_grid.best_params_

now, let's try `RandomizedSearchCV`

In [ ]:
params_dist_grid = {
    'max_depth': [1, 2, 3, 4],
    'gamma': [0, 0.5, 1],
    'n_estimators': randint(1, 1001), # uniform discrete random distribution
    'learning_rate': uniform(), # gaussian distribution
    'subsample': uniform(), # gaussian distribution
    'colsample_bytree': uniform() # gaussian distribution
}

In [ ]:
rs_grid = RandomizedSearchCV(
    estimator=XGBClassifier(**params_fixed, seed=seed),
    param_distributions=params_dist_grid,
    n_iter=10,
    cv=cv,
    scoring='accuracy',
    random_state=seed
)

In [ ]:
bst_grid.fit(X, y)

In [ ]:
bst_grid.best_score_

In [ ]:
bst_grid.best_params_

## **Evaluate Results**

there are many `eval_metric` that you can select from (see more on slides). You can also use custom function

In [ ]:
seed = 123
params = {
    'objective':'binary:logistic',
    'max_depth':1,
    'silent':1,
    'eta':0.5
}
num_rounds = 5

dtrain = xgb.DMatrix('data/agaricus.txt.train')
dtest = xgb.DMatrix('data/agaricus.txt.test')
watchlist  = [(dtest,'test'), (dtrain,'train')]

In [ ]:
params['eval_metric'] = 'error'
bst = xgb.train(params, dtrain, num_rounds, watchlist)

In [ ]:
params['eval_metric'] = 'logloss'
bst = xgb.train(params, dtrain, num_rounds, watchlist)

In [ ]:
params['eval_metric'] = ['logloss', 'auc']
bst = xgb.train(params, dtrain, num_rounds, watchlist)

In [ ]:
def misclassified(pred_probs, dtrain):
    labels = dtrain.get_label() # obtain true labels
    preds = pred_probs > 0.5 # obtain predicted values
    return 'misclassified', np.sum(labels != preds)

In [ ]:
e_results = {}
bst = xgb.train(params, dtrain, num_rounds, watchlist, 
                feval=misclassified, maximize=False, evals_result=e_results)

In [ ]:
e_results

cross validating results

In [ ]:
num_rounds = 10
hist = xgb.cv(params, dtrain, num_rounds, nfold=10, metrics={'error'}, seed=seed)

In [ ]:
hist

Notice that:

- by default we get a pandas data frame object (can be changed with `as_pandas` param),
- metrics are passed as an argument (muliple values are allowed),
- we can use own evaluation metrics (param `feval` and `maximize`),
- we can use early stopping feature (param `early_stopping_rounds`)

## **Early Stopping**

In [ ]:
num_rounds = 1500
bst = xgb.train(params, dtrain, num_rounds, watchlist, early_stopping_rounds=10)

In [ ]:
print(bst.best_score)
print(bst.best_iteration)
print(bst.best_ntree_limit)

## **Dealing with missing values**

In [ ]:
seed = 123
from xgboost.sklearn import XGBClassifier

In [ ]:
data_v = np.random.rand(10,5)
nan_loc = [(2,3), (0,1), (0,2), (1,0), (4,4), (7,2), (9,1)] # nan location
data_m = np.copy(data_v)
for (i, j) in nan_loc:
    data_m[i, j] = np.nan
np.random.seed(seed)
label = np.random.randint(2, size=10)

In [ ]:
# specify general training parameters
params = {
    'objective':'binary:logistic',
    'max_depth':1,
    'silent':1,
    'eta':0.5
}
num_rounds = 5

In [ ]:
dtrain_v = xgb.DMatrix(data_v, label=label)
xgb.cv(params, dtrain_v, num_boost_round=num_rounds, seed=seed)

In [ ]:
dtrain_m = xgb.DMatrix(data_m, label=label, missing=np.nan) # add missing here
xgb.cv(params, dtrain_m, num_rounds, seed=seed)

looks like it works also with missing value. Now, let's try it with scikit-learn wrapper

In [ ]:
from sklearn.cross_validation import cross_val_score

In [ ]:
params = {
    'objective': 'binary:logistic',
    'max_depth': 1,
    'learning_rate': 0.5,
    'silent': 1.0,
    'n_estimators': 5
}
clf = XGBClassifier(**params)

In [ ]:
cross_val_score(clf, data_v, label, cv=2, scoring='accuracy')

In [ ]:
cross_val_score(clf, data_m, label, cv=2, scoring='accuracy')

Both methods works with missing datasets. The Sklearn package by default handles data with `np.nan` as missing

## **Handling imbalance dataset**

In [ ]:
seed = 123

X, y = make_classification(
    n_samples=200,
    n_features=5,
    n_informative=3,
    n_classes=2,
    weights=[.9, .1],
    shuffle=True,
    random_state=seed
)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, 
                                                    stratify=y, 
                                                    random_state=seed)

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)

In [ ]:
params = {
    'objective':'binary:logistic',
    'max_depth':1,
    'silent':1,
    'eta':1
}

num_rounds = 15

In [ ]:
bst = xgb.train(params, dtrain, num_rounds)
y_test_preds = (bst.predict(dtest) > 0.5).astype('int')

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [ ]:
print(accuracy_score(y_test, y_test_preds))
print(precision_score(y_test, y_test_preds))
print(recall_score(y_test, y_test_preds))

We'll add weight to class `1` in this case

In [ ]:
weights = np.zeros(len(y_train))
weights[y_train == 0] = 1
weights[y_train == 1] = 5

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train, weight=weights) # weights added
dtest = xgb.DMatrix(X_test)

In [ ]:
bst = xgb.train(params, dtrain, num_rounds)
y_test_preds = (bst.predict(dtest) > 0.5).astype('int')

In [ ]:
print(accuracy_score(y_test, y_test_preds))
print(precision_score(y_test, y_test_preds))
print(recall_score(y_test, y_test_preds))

use `scale_pos_weight` instead

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)

In [ ]:
train_labels = dtrain.get_label()
ratio = float(np.sum(train_labels == 0)) / np.sum(train_labels == 1)
params['scale_pos_weight'] = ratio

In [ ]:
bst = xgb.train(params, dtrain, num_rounds)
y_test_preds = (bst.predict(dtest) > 0.5).astype('int')

In [ ]:
print(accuracy_score(y_test, y_test_preds))
print(precision_score(y_test, y_test_preds))
print(recall_score(y_test, y_test_preds))

In [ ]:
from IPython.core.display import HTML
HTML(open("../custom.css", "r").read())